### Read me
this code should go to the deleted files folder,
make a csv file of all the deleted files in the folder of all the types


How To use:
get a path to the source folder (folder containing each species with it's deleted 1s segments)
every deleted segment file should be in "ID-segment_number.wav" format.
then just run the notebook

In [5]:
import os
import pandas as pd
from pathlib import Path
from typing import List

### Create Paths to all the folders you have

In [6]:
SourceFolder = Path(r"C:\Users\mhnd0\OneDrive\Desktop\The mosquito project\Mosquito data\deleted data")
paths = [SourceFolder / folder.name for folder in SourceFolder.iterdir()] 
paths

[WindowsPath('C:/Users/mhnd0/OneDrive/Desktop/The mosquito project/Mosquito data/deleted data/ae aegypti'),
 WindowsPath('C:/Users/mhnd0/OneDrive/Desktop/The mosquito project/Mosquito data/deleted data/an arabiensis'),
 WindowsPath('C:/Users/mhnd0/OneDrive/Desktop/The mosquito project/Mosquito data/deleted data/an funestus ss'),
 WindowsPath('C:/Users/mhnd0/OneDrive/Desktop/The mosquito project/Mosquito data/deleted data/culex pipiens complex')]

### Function to get a DataFrame of all the files
Format of dataframe/csv: ID | Type | TimeStamp

In [7]:
#Example file:          200324-Segment_1.wav
def pathsToDataFram(paths: List[Path]) -> pd.DataFrame:
    data = []
    for folder in paths:
        MosquitoType = folder.name
        for file in folder.iterdir():
            if file.is_file() and file.suffix == '.wav':
                try: 
                    fileName = file.stem #full filename
                    #get ID:
                    ID = fileName[ : fileName.index('-')]

                    #get TimeStamp
                    segmentNumber = fileName[fileName.index('_') + 1 : ]
                    timestamp     = f"{int(segmentNumber)-1}-{segmentNumber}"
                    
                    data.append({
                        "id": ID,
                        "Mosquito Species": MosquitoType,
                        "timeStamp": timestamp
                    })
                except Exception as e:
                    print(f"Couldn't process file: {file}; {e}")

    
    return pd.DataFrame(data)

Another function to achive the same thing, however this one makes the csv cleaner.

to be honest i really don't know why it yeilds children in a sorted by name way, maybe it's because I set my os to do this, but I'll use it for easier code lol...

In [8]:
#Example file:          200324-Segment_1.wav
def prettyDataFrame(paths: List[Path]) -> pd.DataFrame:
    data = []
    continuousSegments = [{'id':-1, 'segment':-1, 'type':None}] #list of dictionaries of the continues segments
    #   when we encounter a discintounous segment we add the list and make a new one
    for folder in paths:
        MosquitoType = folder.name
        for file in folder.iterdir():
            if file.is_file() and file.suffix == '.wav':
                try: 
                    fileName = file.stem #full filename
                    #get ID:
                    ID = fileName[ : fileName.index('-')]
                    #get Segment
                    segmentNumber = fileName[fileName.index('_') + 1 : ]
                    
                    IDofContinoutity     = continuousSegments[0]['id']
                    lastContinousSegment = int(continuousSegments[-1]['segment']) + 1
                    
                    #   if discontinoues segment
                    if IDofContinoutity != ID or lastContinousSegment != int(segmentNumber):
                        firstSegment = continuousSegments[0] ['segment']
                        lastSegment  = continuousSegments[-1]['segment']
                        timestamp    = f"[{int(firstSegment)-1}-{lastSegment}]"
                        
                        data.append({
                            "id": continuousSegments[0]['id'],
                            "Mosquito Species": continuousSegments[0]['type'],
                            "timeStamp": timestamp
                        })

                        #re-start the continoutiy
                        continuousSegments = []

                    #   continous segment
                    continuousSegments.append({'id':ID, 'segment': segmentNumber, 'type': MosquitoType})

                    
                except Exception as e:
                    print(f"Couldn't process file: {file}; {e}")



    #   some last cleaning... delete the first initilizer of -1 id
    data = data[1:]
    #   add the last continouity
    firstSegment = continuousSegments[0] ['segment']
    lastSegment  = continuousSegments[-1]['segment']
    timestamp    = f"[{int(firstSegment)-1}-{lastSegment}]"
    
    data.append({
        "id": continuousSegments[0]['id'],
        "Mosquito Species": continuousSegments[0]['type'],
        "timeStamp": timestamp
    })
    return pd.DataFrame(data)

### load the dataframe

In [9]:
df = pathsToDataFram(paths)
df.head(-8)

Couldn't process file: C:\Users\mhnd0\OneDrive\Desktop\The mosquito project\Mosquito data\deleted data\culex pipiens complex\220291_segment0.wav; substring not found


,id,Mosquito Species,timeStamp
0,220291,ae aegypti,0-1
1,220291,ae aegypti,1-2
2,220292,ae aegypti,0-1
3,220292,ae aegypti,14-15
4,220292,ae aegypti,17-18
...,...,...,...
3039,222491,culex pipiens complex,4-5
3040,222491,culex pipiens complex,5-6
3041,222491,culex pipiens complex,6-7
3042,222491,culex pipiens complex,7-8


In [10]:
prettyDf = prettyDataFrame(paths)
prettyDf

Couldn't process file: C:\Users\mhnd0\OneDrive\Desktop\The mosquito project\Mosquito data\deleted data\culex pipiens complex\220291_segment0.wav; substring not found


,id,Mosquito Species,timeStamp
0,220291,ae aegypti,[0-2]
1,220292,ae aegypti,[0-1]
2,220292,ae aegypti,[14-15]
3,220292,ae aegypti,[17-18]
4,220292,ae aegypti,[1-2]
...,...,...,...
1819,222491,culex pipiens complex,[4-9]
1820,222492,culex pipiens complex,[9-10]
1821,222507,culex pipiens complex,[14-16]
1822,222507,culex pipiens complex,[8-9]


### Save to csv

In [11]:
df.to_csv("Cleaned Mosquitoes.csv", index=False)
prettyDf.to_csv("pretty Cleaned Mosquitoes.csv", index=False)